In [10]:
!pip install torch

Defaulting to user installation because normal site-packages is not writeable
DEPRECATION: distro-info 0.23ubuntu1 has a non-standard version number. pip 23.3 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of distro-info or contact the author to suggest that they release a version with a conforming version number. Discussion can be found at https://github.com/pypa/pip/issues/12063
DEPRECATION: python-debian 0.1.36ubuntu1 has a non-standard version number. pip 23.3 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of python-debian or contact the author to suggest that they release a version with a conforming version number. Discussion can be found at https://github.com/pypa/pip/issues/12063
DEPRECATION: tensorflow-gpu 2.11.0 has a non-standard dependency specifier platform_system!="Darwin". pip 23.3 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of tensorfl

In [4]:
import pandas as pd
import os
import requests
import json
import csv
import io
from collections import defaultdict
from time import time
from sklearn import metrics
from sklearn.feature_extraction.text import TfidfVectorizer
#from flair.data import Sentence
#from flair.nn import Classifier
from chembl_webresource_client.new_client import new_client
from sqlalchemy import create_engine
from sqlalchemy import text

# 1. Get targets from disease

In [5]:
"""
insert code here that
1. Takes an EFO as input
2. displays a dataframe of targets for that disease
3. uploads that dataframe to a table in planetscale called "disease_to_target"

"""
from sqlalchemy import create_engine
from dotenv import dotenv_values
from sqlalchemy import text
config = dotenv_values('database_url.env')
url = config['DATABASE_URL']

engine = create_engine(url, echo=False)



## Set disease_id variable for desired disease

disease_id = "EFO_0005537"


with engine.begin() as conn:
    query = text("SELECT * FROM disease_to_target WHERE disease_id = '{disease_id}';".format(disease_id = disease_id))
    disease_df = pd.read_sql(query, conn)

display(disease_df)


target_ids = disease_df.sort_values(by = ['association_score'], ascending=False)['target_ensemble_id'].values
print(target_ids)
target_id = target_ids[1]

print(target_id)
print(type(target_id))

,level_0,index,disease_id,target_ensemble_id,target_chembl_id,association_score
0,0,0,EFO_0005537,ENSG00000184292,CHEMBL3856163,0.431490
1,1,1,EFO_0005537,ENSG00000198900,CHEMBL1781,0.424207
2,2,2,EFO_0005537,ENSG00000120217,CHEMBL3580522,0.405569
3,3,3,EFO_0005537,ENSG00000258947,CHEMBL2597,0.402827
4,4,4,EFO_0005537,ENSG00000137267,CHEMBL3797012,0.399973
...,...,...,...,...,...,...
5012,5012,5012,EFO_0005537,ENSG00000135164,CHEMBL3707469,0.001478
5013,5013,5013,EFO_0005537,ENSG00000115665,CHEMBL4507,0.001478
5014,5014,5014,EFO_0005537,ENSG00000146109,CHEMBL4507,0.001478
5015,5015,5015,EFO_0005537,ENSG00000225830,CHEMBL4507,0.001478


# 2. Get compounds and assays

In [6]:
"""
insert code here that
1. takes a target as input
2. searches the database for compounds and assays for that target
3. displays a dataframe with at least these columns: target id, compound id, assay id
"""
engine = create_engine(url, echo=False)
#target_id = 'ENSG00000120217'


with engine.begin() as conn:
    query = text("SELECT * FROM target_to_compounds WHERE target_ensemble_id='{target}';".format(target=target_id))
    target_to_compounds_df = pd.read_sql(query, conn)

display(target_to_compounds_df)



,level_0,index,compound_id,smiles,type,standard_value,assay_id,assay_type,assay_description,document_id,abstract,target_ensemble_id
0,1895,0,CHEMBL4776444,Cc1c(COc2cc(OCc3cncc(C#N)c3)c(CNC(C)(C)C(=O)O)...,IC50,1.0,CHEMBL4775268,B,Binding affinity to human PDL1 assessed as red...,CHEMBL4765466,Therapeutic interference of the programmed cel...,ENSG00000120217
1,1896,1,CHEMBL4081869,COc1cc(OCc2cccc(-c3ccccc3)c2C)cc(OC)c1CNCCNC(C)=O,IC50,1.0,CHEMBL4775269,B,Binding affinity to PDL1 (unknown origin) asse...,CHEMBL4765466,Therapeutic interference of the programmed cel...,ENSG00000120217
2,1897,2,CHEMBL4776700,Cn1cnc(S(=O)(=O)N2CCCC3=C[C@H]4C[C@H](CN5CCCC[...,IC50,1.0,CHEMBL4775268,B,Binding affinity to human PDL1 assessed as red...,CHEMBL4765466,Therapeutic interference of the programmed cel...,ENSG00000120217
3,1898,3,CHEMBL4785255,Cc1c(COc2cc(OCc3cncc(C#N)c3)c(CN[C@](C)(CO)C(=...,IC50,1.0,CHEMBL4775268,B,Binding affinity to human PDL1 assessed as red...,CHEMBL4765466,Therapeutic interference of the programmed cel...,ENSG00000120217
4,1899,4,CHEMBL5171101,CCCC[C@H]1C(=O)N(C)[C@@H](CCCC)C(=O)N[C@@H](C)...,IC50,1.03,CHEMBL5096762,B,Inhibition of PD-L1 (unknown origin) by HTRF a...,CHEMBL5096155,Tumor immunotherapy has made great progress in...,ENSG00000120217
5,1900,5,CHEMBL4071326,Cc1c(COc2cc(OCc3cccc(C#N)c3)c(CN3C[C@H](O)C[C@...,IC50,1.4,CHEMBL4017391,B,Binding affinity to His-tagged PD-L1 (unknown ...,CHEMBL4014356,Blockade of the PD-1/PD-L1 immune checkpoint p...,ENSG00000120217
6,1901,6,CHEMBL4084368,Cc1cc(CN[C@H](CO)C(=O)O)c(OCc2cccc(C#N)c2)cc1O...,IC50,2.25,CHEMBL4017391,B,Binding affinity to His-tagged PD-L1 (unknown ...,CHEMBL4014356,Blockade of the PD-1/PD-L1 immune checkpoint p...,ENSG00000120217
7,1902,7,CHEMBL4288470,CCCC[C@H]1C(=O)N(C)[C@@H](CCCC)C(=O)N[C@@H](CC...,IC50,5.2,CHEMBL4263391,B,Inhibition of PD-L1 (unknown origin),CHEMBL4261578,The modulation of immune checkpoint receptors ...,ENSG00000120217
8,1903,8,CHEMBL3582254,Cc1c(COc2ccc(CN[C@H](C)C(=O)O)cc2Cl)cccc1-c1cc...,IC50,6.0,CHEMBL3583018,B,Binding affinity to His-tagged PD-L1 (unknown ...,CHEMBL3580542,,ENSG00000120217
9,1904,9,CHEMBL4061613,COc1cc(OCc2cccc(-c3ccccc3)c2C)cc(OC)c1CN[C@@H]...,IC50,6.0,CHEMBL4017391,B,Binding affinity to His-tagged PD-L1 (unknown ...,CHEMBL4014356,Blockade of the PD-1/PD-L1 immune checkpoint p...,ENSG00000120217


# 3. Propose a new compound

In [7]:
# need to tell deepchem where to find cuda

In [8]:
%env XLA_FLAGS=--xla_gpu_cuda_data_dir=/usr/lib/cuda

env: XLA_FLAGS=--xla_gpu_cuda_data_dir=/usr/lib/cuda


In [12]:
"""
insert code here that
1. trains a model to predict ic50 values for a compound on a given target
2. downloads ALL the compounds in the database (independent of target)
3. runs the model to predict ic50 values for each compound in the database
4. displays a dataframe that has the following two columns : compound, predicted ic50
5. prints out the compound that has the highest ic50 value that has NOT been tested on the target already.
"""
# 1. Train Model
import deepchem as dc
import numpy as np

import os

import tensorflow as tf
from tensorflow import keras

print(tf.version.VERSION)

#!git clone https://github.com/dsalinasduron-msmary/chemical_informatics.git

import os
import pickle
#os.listdir()
#os.chdir('chemical_informatics')
#os.getcwd()
"""
f = open('ENSG00000128191dataframe.pickle','rb')
compound_dataset = pickle.load(f)
f.close()
display(compound_dataset)
"""
compound_dataset = target_to_compounds_df
smiles = compound_dataset['smiles']
IC50 = compound_dataset['standard_value']
featurizer = dc.feat.ConvMolFeaturizer()
compound_dataset['featurized'] = featurizer.featurize(smiles)
compound_dataset['divided values'] = compound_dataset['standard_value'].astype(float).div(108000)
compound_dataset['pIC50'] = np.log10(compound_dataset['divided values'].astype(float)).mul(-1)
compound_dataset['number'] = list(range(0,len(compound_dataset)))
display(compound_dataset)

#compound_dataset['pIC50'].hist()
"""
x = compound_dataset
f = open('x.pickle','wb')
pickle.dump(x,f)
f.close()
"""
import pandas as pd
training_dataset = compound_dataset.sample(frac = 0.7)

#training_dataset.featurized[0].n_feat

testing_dataset = (compound_dataset[~compound_dataset['number'].isin(training_dataset['number'])])
display(testing_dataset)

numpy_training_dataset = dc.data.NumpyDataset(X=training_dataset['featurized'],y=training_dataset['pIC50'].astype(float), ids=training_dataset['smiles'])
numpy_testing_dataset = dc.data.NumpyDataset(X=testing_dataset['featurized'],y=testing_dataset['pIC50'].astype(float), ids=testing_dataset['smiles'])
display(numpy_training_dataset)
display(numpy_testing_dataset)

model = dc.models.GraphConvModel(n_tasks=1, mode='regression', dropout=0.2, dense_layer_size=10)

model.fit(numpy_training_dataset, nb_epoch=5000)

#model = dc.models.GraphConvModel(n_tasks=1, mode='regression', dropout=0.2, dense_layer_size=10, model_dir="./compund_model")
#model.restore()

metric = dc.metrics.Metric(dc.metrics.pearson_r2_score)
print("Training set score:", model.evaluate(numpy_training_dataset, [metric]))
print("Test set score:", model.evaluate(numpy_testing_dataset, [metric]))


#2. Download all compounds in database
with engine.begin() as conn:
    query = text("SELECT compound_id,smiles FROM target_to_compounds;")
    compounds_df = pd.read_sql(query, conn)

predict_list = []
for smile in range(0, len(compounds_df)):
    predict_list.append(None)
print(len(predict_list))

compounds_df['predicted_pIC50'] = predict_list

curated_compounds_df = compounds_df.dropna(subset=['smiles'])

new_smiles = curated_compounds_df['smiles']
curated_compounds_df['featurized'] = featurizer.featurize(new_smiles)

#3. run the model to predict ic50 values for each compound in the database
dataset = dc.data.NumpyDataset(X=curated_compounds_df['featurized'], y=curated_compounds_df['predicted_pIC50'], ids=curated_compounds_df['smiles'])
curated_compounds_df['predicted_pIC50'] = model.predict(dataset)

# 4. displays a dataframe that has the following two columns : compound, predicted ic50
display(curated_compounds_df)

# 5. print out the compound that has the highest ic50 value that has NOT been tested on the target already.

curated_compounds_no_target = (curated_compounds_df[~curated_compounds_df['smiles'].isin(target_to_compounds_df['smiles'])])

print(len(curated_compounds_no_target) <= len(curated_compounds_df) - len(target_to_compounds_df))


df = curated_compounds_no_target.sort_values(by = ['predicted_pIC50'], ascending=False)
display(df)
compound_ids = df['compound_id'].values
compound_smiles = df['smiles'].values

compound_ids_and_smiles = list(zip(compound_ids, compound_smiles))
best_compound = (compound_ids_and_smiles[0])

# 6. get InChI key for top compound and generate a url for Zinc

url = "https://cactus.nci.nih.gov/chemical/structure/{smiles}/stdinchikey".format(smiles = best_compound[1])
r = requests.get(url=url)
raw_inchikey = r.text
inchikey = raw_inchikey.split('=')[1]

print(inchikey)

zinc_url = 'https://zinc15.docking.org/substances/?inchikey={inchikey}'.format(inchikey = inchikey)
print(zinc_url)



2.13.0


,level_0,index,compound_id,smiles,type,standard_value,assay_id,assay_type,assay_description,document_id,abstract,target_ensemble_id,featurized,divided values,pIC50,number
0,1895,0,CHEMBL4776444,Cc1c(COc2cc(OCc3cncc(C#N)c3)c(CNC(C)(C)C(=O)O)...,IC50,1.0,CHEMBL4775268,B,Binding affinity to human PDL1 assessed as red...,CHEMBL4765466,Therapeutic interference of the programmed cel...,ENSG00000120217,<deepchem.feat.mol_graphs.ConvMol object at 0x...,0.000009,5.033424,0
1,1896,1,CHEMBL4081869,COc1cc(OCc2cccc(-c3ccccc3)c2C)cc(OC)c1CNCCNC(C)=O,IC50,1.0,CHEMBL4775269,B,Binding affinity to PDL1 (unknown origin) asse...,CHEMBL4765466,Therapeutic interference of the programmed cel...,ENSG00000120217,<deepchem.feat.mol_graphs.ConvMol object at 0x...,0.000009,5.033424,1
2,1897,2,CHEMBL4776700,Cn1cnc(S(=O)(=O)N2CCCC3=C[C@H]4C[C@H](CN5CCCC[...,IC50,1.0,CHEMBL4775268,B,Binding affinity to human PDL1 assessed as red...,CHEMBL4765466,Therapeutic interference of the programmed cel...,ENSG00000120217,<deepchem.feat.mol_graphs.ConvMol object at 0x...,0.000009,5.033424,2
3,1898,3,CHEMBL4785255,Cc1c(COc2cc(OCc3cncc(C#N)c3)c(CN[C@](C)(CO)C(=...,IC50,1.0,CHEMBL4775268,B,Binding affinity to human PDL1 assessed as red...,CHEMBL4765466,Therapeutic interference of the programmed cel...,ENSG00000120217,<deepchem.feat.mol_graphs.ConvMol object at 0x...,0.000009,5.033424,3
4,1899,4,CHEMBL5171101,CCCC[C@H]1C(=O)N(C)[C@@H](CCCC)C(=O)N[C@@H](C)...,IC50,1.03,CHEMBL5096762,B,Inhibition of PD-L1 (unknown origin) by HTRF a...,CHEMBL5096155,Tumor immunotherapy has made great progress in...,ENSG00000120217,<deepchem.feat.mol_graphs.ConvMol object at 0x...,0.000010,5.020587,4
5,1900,5,CHEMBL4071326,Cc1c(COc2cc(OCc3cccc(C#N)c3)c(CN3C[C@H](O)C[C@...,IC50,1.4,CHEMBL4017391,B,Binding affinity to His-tagged PD-L1 (unknown ...,CHEMBL4014356,Blockade of the PD-1/PD-L1 immune checkpoint p...,ENSG00000120217,<deepchem.feat.mol_graphs.ConvMol object at 0x...,0.000013,4.887296,5
6,1901,6,CHEMBL4084368,Cc1cc(CN[C@H](CO)C(=O)O)c(OCc2cccc(C#N)c2)cc1O...,IC50,2.25,CHEMBL4017391,B,Binding affinity to His-tagged PD-L1 (unknown ...,CHEMBL4014356,Blockade of the PD-1/PD-L1 immune checkpoint p...,ENSG00000120217,<deepchem.feat.mol_graphs.ConvMol object at 0x...,0.000021,4.681241,6
7,1902,7,CHEMBL4288470,CCCC[C@H]1C(=O)N(C)[C@@H](CCCC)C(=O)N[C@@H](CC...,IC50,5.2,CHEMBL4263391,B,Inhibition of PD-L1 (unknown origin),CHEMBL4261578,The modulation of immune checkpoint receptors ...,ENSG00000120217,<deepchem.feat.mol_graphs.ConvMol object at 0x...,0.000048,4.317420,7
8,1903,8,CHEMBL3582254,Cc1c(COc2ccc(CN[C@H](C)C(=O)O)cc2Cl)cccc1-c1cc...,IC50,6.0,CHEMBL3583018,B,Binding affinity to His-tagged PD-L1 (unknown ...,CHEMBL3580542,,ENSG00000120217,<deepchem.feat.mol_graphs.ConvMol object at 0x...,0.000056,4.255273,8
9,1904,9,CHEMBL4061613,COc1cc(OCc2cccc(-c3ccccc3)c2C)cc(OC)c1CN[C@@H]...,IC50,6.0,CHEMBL4017391,B,Binding affinity to His-tagged PD-L1 (unknown ...,CHEMBL4014356,Blockade of the PD-1/PD-L1 immune checkpoint p...,ENSG00000120217,<deepchem.feat.mol_graphs.ConvMol object at 0x...,0.000056,4.255273,9


,level_0,index,compound_id,smiles,type,standard_value,assay_id,assay_type,assay_description,document_id,abstract,target_ensemble_id,featurized,divided values,pIC50,number
4,1899,4,CHEMBL5171101,CCCC[C@H]1C(=O)N(C)[C@@H](CCCC)C(=O)N[C@@H](C)...,IC50,1.03,CHEMBL5096762,B,Inhibition of PD-L1 (unknown origin) by HTRF a...,CHEMBL5096155,Tumor immunotherapy has made great progress in...,ENSG00000120217,<deepchem.feat.mol_graphs.ConvMol object at 0x...,0.000010,5.020587,4
5,1900,5,CHEMBL4071326,Cc1c(COc2cc(OCc3cccc(C#N)c3)c(CN3C[C@H](O)C[C@...,IC50,1.4,CHEMBL4017391,B,Binding affinity to His-tagged PD-L1 (unknown ...,CHEMBL4014356,Blockade of the PD-1/PD-L1 immune checkpoint p...,ENSG00000120217,<deepchem.feat.mol_graphs.ConvMol object at 0x...,0.000013,4.887296,5
7,1902,7,CHEMBL4288470,CCCC[C@H]1C(=O)N(C)[C@@H](CCCC)C(=O)N[C@@H](CC...,IC50,5.2,CHEMBL4263391,B,Inhibition of PD-L1 (unknown origin),CHEMBL4261578,The modulation of immune checkpoint receptors ...,ENSG00000120217,<deepchem.feat.mol_graphs.ConvMol object at 0x...,0.000048,4.317420,7
15,1910,15,CHEMBL4089730,COc1nc(OCc2cccc(-c3ccccc3)c2C)ccc1CNCCNC(C)=O,IC50,18.0,CHEMBL4775269,B,Binding affinity to PDL1 (unknown origin) asse...,CHEMBL4765466,Therapeutic interference of the programmed cel...,ENSG00000120217,<deepchem.feat.mol_graphs.ConvMol object at 0x...,0.000167,3.778151,15
18,1913,18,CHEMBL4089730,COc1nc(OCc2cccc(-c3ccccc3)c2C)ccc1CNCCNC(C)=O,IC50,18.0,CHEMBL4017391,B,Binding affinity to His-tagged PD-L1 (unknown ...,CHEMBL4014356,Blockade of the PD-1/PD-L1 immune checkpoint p...,ENSG00000120217,<deepchem.feat.mol_graphs.ConvMol object at 0x...,0.000167,3.778151,18
22,1917,22,CHEMBL3582253,Cc1cc(CN2CCC(C(=O)NCCO)CC2)ccc1OCc1cccc(-c2ccc...,IC50,110.0,CHEMBL3583018,B,Binding affinity to His-tagged PD-L1 (unknown ...,CHEMBL3580542,,ENSG00000120217,<deepchem.feat.mol_graphs.ConvMol object at 0x...,0.001019,2.992031,22
27,1922,27,CHEMBL4777505,CC[C@H](C)[C@H](NC(=O)[C@H](C)NC(=O)CN)C(=O)N[...,IC50,1560.0,CHEMBL4688873,B,Antagonist activity at human PDL1 assessed as ...,CHEMBL4680328,Blocking the interaction of programmed cell de...,ENSG00000120217,<deepchem.feat.mol_graphs.ConvMol object at 0x...,0.014444,1.840299,27
29,1924,29,CHEMBL4791550,CC[C@H](C)[C@H](NC(=O)[C@H](C)NC(=O)CN)C(=O)N[...,IC50,5600.0,CHEMBL4688873,B,Antagonist activity at human PDL1 assessed as ...,CHEMBL4680328,Blocking the interaction of programmed cell de...,ENSG00000120217,<deepchem.feat.mol_graphs.ConvMol object at 0x...,0.051852,1.285236,29
32,1927,32,CHEMBL4790749,CC[C@H](C)[C@H](NC(=O)[C@H](C)NC(=O)CNC(=O)[C@...,IC50,80700.0,CHEMBL4688873,B,Antagonist activity at human PDL1 assessed as ...,CHEMBL4680328,Blocking the interaction of programmed cell de...,ENSG00000120217,<deepchem.feat.mol_graphs.ConvMol object at 0x...,0.747222,0.126550,32
33,1928,33,CHEMBL4791798,CC[C@H](C)[C@H](NC(=O)[C@H](C)NC(=O)CNC(=O)[C@...,IC50,93500.0,CHEMBL4688873,B,Antagonist activity at human PDL1 assessed as ...,CHEMBL4680328,Blocking the interaction of programmed cell de...,ENSG00000120217,<deepchem.feat.mol_graphs.ConvMol object at 0x...,0.865741,0.062612,33


<NumpyDataset X.shape: (24,), y.shape: (24,), w.shape: (24,), ids: ['Cc1cc(CN[C@H](CO)C(=O)O)c(OCc2cccc(C#N)c2)cc1OCc1cccc(-c2ccc3c(c2)OCCO3)c1C'
 'COc1cc(OCc2cccc(-c3ccccc3)c2C#N)cc(OC)c1CN1CCCC[C@H]1C(=O)O'
 'C[C@@H](O)[C@H](NCc1cc(Cl)c(OCc2cccc(-c3ccccc3)c2Br)cc1OCC1CC1)C(=O)O'
 ... 'CC(O)[C@H](NC(=O)N[C@@H](CC(N)=O)c1nnc([C@@H](N)CO)s1)C(=O)O'
 'CC[C@H](C)[C@H](NC(=O)[C@H](C)NC(=O)CNC(=O)[C@@H](N)CS)C(=O)N[C@@H](CO)C(=O)N[C@@H](CC(C)C)C(=O)N[C@@H](C)C(=O)N1CCC[C@H]1C(=O)N[C@@H](CCCCN)C(=O)N[C@@H](C)C(=O)N[C@@H](CCC(N)=O)C(=O)N[C@H](C(=O)N[C@@H](CCCCN)C(=O)N[C@@H](CCC(=O)O)C(=O)N[C@@H](CO)C(=O)O)[C@@H](C)CC'
 'O=C(/C=C/c1ccc(O)c(O)c1)O[C@H]1C[C@@](O)(C(=O)O)C[C@H](O)[C@@H]1O'], task_names: [0]>

<NumpyDataset X.shape: (10,), y.shape: (10,), w.shape: (10,), ids: ['CCCC[C@H]1C(=O)N(C)[C@@H](CCCC)C(=O)N[C@@H](C)C(=O)N[C@H](C(=O)NCC(N)=O)CSCC(=O)N[C@@H](Cc2ccc(O)cc2)CC(=O)N2CCCC[C@H]2C(=O)N[C@@H](C(N)=O)C(=O)N2CC[C@@H](C2)C(=O)N[C@@H](Cc2c[nH]cn2)C(=O)N[C@@H](CCC(N)=O)C(=O)N2C[C@H](O)C[C@H]2C(=O)N[C@@H](Cc2cc3ccccc3[nH]2)C(=O)N[C@@H](CO)C(=O)N[C@@H](Cc2c[nH]c3ccccc23)C(=O)N1C'
 'Cc1c(COc2cc(OCc3cccc(C#N)c3)c(CN3C[C@H](O)C[C@@H]3C(=O)O)cc2Cl)cccc1-c1ccc2c(c1)OCCO2'
 'CCCC[C@H]1C(=O)N(C)[C@@H](CCCC)C(=O)N[C@@H](CCCNC(=N)N)C(=O)NC(C(=O)NCC(N)=O)CSCC(=O)N[C@@H](Cc2ccccc2)C(=O)N(C)[C@@H](C)C(=O)N[C@@H](CC(N)=O)C(=O)N2CCCC2C(=O)N[C@@H](CC(C)C)C(=O)NC(=O)N(C)CC(=O)NCC(=O)N[C@@H](Cc2c[nH]c3ccccc23)C(=O)N[C@@H](Cc2c[nH]c3ccccc23)C(=O)N1C'
 'COc1nc(OCc2cccc(-c3ccccc3)c2C)ccc1CNCCNC(C)=O'
 'COc1nc(OCc2cccc(-c3ccccc3)c2C)ccc1CNCCNC(C)=O'
 'Cc1cc(CN2CCC(C(=O)NCCO)CC2)ccc1OCc1cccc(-c2ccccc2)c1C'
 'CC[C@H](C)[C@H](NC(=O)[C@H](C)NC(=O)CN)C(=O)N[C@@H](CO)C(=O)N[C@@H](CC(C)C)C(=O)N[C@@H](C)C(=O)N1C

FailedPreconditionError: Graph execution error:

Detected at node 'Adam/StatefulPartitionedCall_93' defined at (most recent call last):
    File "/usr/lib/python3.8/runpy.py", line 194, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "/usr/lib/python3.8/runpy.py", line 87, in _run_code
      exec(code, run_globals)
    File "/usr/lib/python3/dist-packages/ipykernel_launcher.py", line 16, in <module>
      app.launch_new_instance()
    File "/home/ubuntu/.local/lib/python3.8/site-packages/traitlets/config/application.py", line 1043, in launch_instance
      app.start()
    File "/usr/lib/python3/dist-packages/ipykernel/kernelapp.py", line 583, in start
      self.io_loop.start()
    File "/home/ubuntu/.local/lib/python3.8/site-packages/tornado/platform/asyncio.py", line 195, in start
      self.asyncio_loop.run_forever()
    File "/usr/lib/python3.8/asyncio/base_events.py", line 570, in run_forever
      self._run_once()
    File "/usr/lib/python3.8/asyn

# 4. Propose an assay

In [ ]:
#!pip install seaborn

In [ ]:
import seaborn
from matplotlib.pyplot import hist
from sklearn.neighbors import KernelDensity
from scipy import stats
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.cluster import AgglomerativeClustering
import numpy as np
from scipy.cluster.hierarchy import dendrogram
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
import pandas as pd

In [ ]:
"""
insert code here that
1. retrieves the terms for all the assays that are relevant to the target the user picked.
2. clusters the assays according to their descriptive terms
3. plots the clusters (set n_clusters = 10)
4. prints out the title of one assay from each cluster.
"""

config = dotenv_values("database_URL.env")
url = config['DATABASE_URL']
engine = create_engine(url, echo=False)

with engine.begin() as conn:
    query = text("select * from target_to_compounds;")
    target_list = pd.read_sql(query, conn)
display(target_list)


three_col_list = target_list[['assay_id', 'assay_description', 'abstract']].copy()

unique_list = []
for index, row in three_col_list.iterrows():
    if (row["assay_id"], row["assay_description"], row["abstract"]) in unique_list:
        continue
    else:
        unique_list.append((row["assay_id"], row["assay_description"], row["abstract"]))

Assay_Descriptions = [abstract for (assay_ids, assay_name, abstract) in unique_list]
Assay_Descriptions_Joined = ':: '.join(Assay_Descriptions)

print(len(unique_list))
#print(unique_list)
#print(Assay_Descriptions)


tagger = Classifier.load('hunflair')
print("Finished setting tagger")

Assay_Descriptions_List = []

sentence = Sentence(Assay_Descriptions_Joined)
tagger.predict(sentence)

Assay_Key_Words = ""
for label in sentence.get_labels():
    Assay_Key_Words += label.data_point.text
    Assay_Key_Words += " "
    #print(label.data_point.text)
    #print(Assay_Key_Words)

#adds the Descriptions of the Assays to the Assay Description List
Assay_Descriptions_List.append(Assay_Key_Words)

def tokenize_with_flair(description):
    sentence = Sentence(description)
    tagger.predict(sentence)
    return [ l.data_point.text for l in sentence.get_labels() ]



Assay_Count_Vect = CountVectorizer(tokenizer=tokenize_with_flair)
Assay_Train_Counts = Assay_Count_Vect.fit_transform(Assay_Descriptions_List)

n_clusters = 10  # number of regions
X = Assay_Train_Counts.toarray()

ward = AgglomerativeClustering(
    n_clusters = n_clusters, linkage="ward", connectivity=None, compute_full_tree= True,compute_distances= True
)
ward.fit(X)

print(ward.distances_)
print(ward.distances_.shape)

X_Hist = ward.distances_
#hist(X_Hist)
kernel = stats.gaussian_kde(X_Hist)
print(kernel(X_Hist))

#first dendogram
seaborn.clustermap(X,method='ward')

tfidf_transformer=TfidfTransformer(smooth_idf=True,use_idf=True)
tfidf_transformer.fit(Assay_Train_Counts)


# print idf values
df_idf = pd.DataFrame(tfidf_transformer.idf_,index=Assay_Count_Vect.get_feature_names(),columns=["idf_weights"])

# sort ascending
df_idf.sort_values(by=['idf_weights'])

# count matrix
count_vector=Assay_Count_Vect.transform(Assay_Descriptions_List)

# tf-idf scores
tf_idf_vector=tfidf_transformer.transform(count_vector)
print(tf_idf_vector)
tf_idf_vector.shape

feature_names = Assay_Count_Vect.get_feature_names()

#get tfidf vector for first document
first_document_vector=tf_idf_vector[0]

#print the scores
df = pd.DataFrame(first_document_vector.T.todense(), index=feature_names, columns=["tfidf"])
df.sort_values(by=["tfidf"],ascending=False)


#second dendogram
seaborn.clustermap(tf_idf_vector.toarray(),method='ward')

Finished Installing flair
200
done
ENSG00000184292
0
0
ENSG00000198900
456
456
ENSG00000120217
34
490


,target_ENSG,compound_id,smiles,type,standard_value,assay_id,assay_type,assay_description,document_ID,abstract
0,ENSG00000198900,CHEMBL3968023,C[n+]1ccc2c3c(nc4cc5c(cc42)OCO5)-c2ccccc2-c31....,IC50,0.45,CHEMBL3867357,B,Inhibition of human DNA topoisomerase 1 using ...,CHEMBL3865847,"Eupolauridine, an indenonaphthyridine alkaloid..."
1,ENSG00000198900,CHEMBL3914791,C[n+]1c2c3c(nc(-c4ccccc4)cc3c3ccccc31)-c1ccccc...,IC50,0.475,CHEMBL3867357,B,Inhibition of human DNA topoisomerase 1 using ...,CHEMBL3865847,"Eupolauridine, an indenonaphthyridine alkaloid..."
2,ENSG00000198900,CHEMBL3896867,[Br-].c1ccc(C[n+]2ccc3c4c(nc5cc6c(cc53)OCO6)-c...,IC50,0.52,CHEMBL3867357,B,Inhibition of human DNA topoisomerase 1 using ...,CHEMBL3865847,"Eupolauridine, an indenonaphthyridine alkaloid..."
3,ENSG00000198900,CHEMBL3889816,[Br-].c1ccc(C[n+]2ccc3c4c(nc5ccccc53)-c3ccccc3...,IC50,0.57,CHEMBL3867357,B,Inhibition of human DNA topoisomerase 1 using ...,CHEMBL3865847,"Eupolauridine, an indenonaphthyridine alkaloid..."
4,ENSG00000198900,CHEMBL3984603,Cc1cc2c3c(nc4ccccc42)-c2ccccc2-c3[n+]1C.[I-],IC50,0.7,CHEMBL3867357,B,Inhibition of human DNA topoisomerase 1 using ...,CHEMBL3865847,"Eupolauridine, an indenonaphthyridine alkaloid..."
...,...,...,...,...,...,...,...,...,...,...
485,ENSG00000120217,CHEMBL4791550,CC[C@H](C)[C@H](NC(=O)[C@H](C)NC(=O)CN)C(=O)N[...,IC50,5600.0,CHEMBL4688873,B,Antagonist activity at human PDL1 assessed as ...,CHEMBL4680328,Blocking the interaction of programmed cell de...
486,ENSG00000120217,CHEMBL4798338,CC[C@H](C)[C@H](NC(=O)[C@H](C)NC(=O)CNC(=O)[C@...,IC50,19200.0,CHEMBL4688873,B,Antagonist activity at human PDL1 assessed as ...,CHEMBL4680328,Blocking the interaction of programmed cell de...
487,ENSG00000120217,CHEMBL366760,O=C(/C=C/c1ccc(O)c(O)c1)O[C@H]1C[C@@](O)(C(=O)...,IC50,36560.0,CHEMBL4613347,B,Antagonist activity against PDL1 (unknown origin),CHEMBL4613205,Blockade the interaction of the programmed cel...
488,ENSG00000120217,CHEMBL4790749,CC[C@H](C)[C@H](NC(=O)[C@H](C)NC(=O)CNC(=O)[C@...,IC50,80700.0,CHEMBL4688873,B,Antagonist activity at human PDL1 assessed as ...,CHEMBL4680328,Blocking the interaction of programmed cell de...


Finished setting tagger
